In [3]:
#!pip install tqdm

In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import csv
import GPy
import numpy as np
import pickle

from datetime import datetime
from glob import glob
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm, percentileofscore
from tqdm import tqdm



warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


# Load raw dataset

In [2]:
raw_meters_path = "..\\data\\meters\\raw\\"

# files in directory
files = glob(raw_meters_path + "*.csv")

In [3]:

dfs = [] # empty list of the dataframes to create
for file in files: # for each file in directory
    meter_type = file.split("\\")[4].split(".")[0] # meter_type to rename the value feature
    meter = pd.read_csv(file) # load the dataset
    meter = pd.melt(meter, id_vars = "timestamp", var_name = "building_id", value_name = "meter_reading") # melt dataset
    meter["meter"] = str(meter_type) # adds column with the meter type
    dfs.append(meter) # append to list
raw_data = pd.concat(dfs, axis=0, ignore_index=True) # concatenate all meter
del(dfs, meter, file, files, meter_type)

raw_data.head()


,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Panther_office_Clementine,NaN,chilledwater
1,2016-01-01 01:00:00,Panther_office_Clementine,NaN,chilledwater
2,2016-01-01 02:00:00,Panther_office_Clementine,NaN,chilledwater
3,2016-01-01 03:00:00,Panther_office_Clementine,NaN,chilledwater
4,2016-01-01 04:00:00,Panther_office_Clementine,NaN,chilledwater


# Load cleaned dataset

In [4]:
cleaned_meters_path = "..\\data\\meters\\cleaned\\"

# files in directory
files = glob(cleaned_meters_path + "*.csv")


In [5]:
dfs = [] # empty list of the dataframes to create
for file in files: # for each file in directory
    meter_type = file.split("\\")[4].split(".")[0] # meter_type to rename the value feature
    meter = pd.read_csv(file) # load the dataset
    meter = pd.melt(meter, id_vars = "timestamp", var_name = "building_id", value_name = "meter_reading") # melt dataset
    meter["meter"] = str(meter_type) # adds column with the meter type
    dfs.append(meter) # append to list
complete_data_cleaned = pd.concat(dfs, axis=0, ignore_index=True) # concatenate all meter
del(dfs, meter, file, files, meter_type)

complete_data_cleaned.head()


,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
1,2016-01-01 01:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
2,2016-01-01 02:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
3,2016-01-01 03:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
4,2016-01-01 04:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned


In [6]:
# Note this cell might take some time to finish

# Convert timestamp field from string into pd.datetime object
complete_data_cleaned['timestamp'] = pd.to_datetime(complete_data_cleaned['timestamp'])

# Add column indicating the year, month and dayOfTheWeek for that timestamp
complete_data_cleaned['date'] = complete_data_cleaned['timestamp'].dt.date
complete_data_cleaned['year'] = complete_data_cleaned['timestamp'].dt.year
complete_data_cleaned['month'] = complete_data_cleaned['timestamp'].dt.month
complete_data_cleaned['dayOfWeek'] = complete_data_cleaned['timestamp'].dt.dayofweek


MemoryError: Unable to allocate 409. MiB for an array with shape (53561832,) and data type int64

# Load benchmarks

In [ ]:
cleaned_meters_path = "..\\data\\"

# files in directory
files = glob(cleaned_meters_path + "*.csv")


benchmark = pd.read_csv(files[0]) # load the dataset

benchmark

In [ ]:
buildingNames = benchmark['name'].unique()

# Specify constants

In [ ]:
# We load the first 
building_name = buildingNames[0]
representative_df = complete_data_cleaned.loc[(complete_data_cleaned['building_id'] == building_name)
                                               & (complete_data_cleaned['meter'] == 'electricity_cleaned')]

In [ ]:
TRAIN_TEST_SPLIT = 0.3
DATAPOINS_PER_BUILDING_AND_METER_TYPE = representative_df.shape[0]
SPLIT_INDEX = int(DATAPOINS_PER_BUILDING_AND_METER_TYPE * (1 - TRAIN_TEST_SPLIT))
SPLIT_TIMESTAMP =  representative_df.iloc[SPLIT_INDEX]['timestamp']
TRAIN_DIMENSIONS = 5
SAMPLE_SIZE = 2000
DATAPOINTS_ONE_WEEK = 24 * 7

# Intermediate computations and ordering of dataframes

In [ ]:
# This cell might take some time to finish

# Group the DataFrame by 'building_id' and 'meter_type'
complete_data_grouped = complete_data_cleaned.groupby(['building_id', 'meter'])

# Initialize an empty list to store the smaller DataFrames
dfs = []
df_keys = []

# Iterate over the groups and create smaller DataFrames
for group_key, group in complete_data_grouped:
    df_keys.append(group_key)
    dfs.append(group.copy()) 

In [ ]:
group_keys = np.array(df_keys)

In [ ]:
dfs_train = []
dfs_test = []

for df in dfs: 
    
    temp_train_df, temp_test_df = split_df_to_train_and_test(df, SPLIT_TIMESTAMP)
    dfs_train.append(temp_train_df)
    dfs_test.append(temp_test_df)
        

# Helper functions

In [ ]:
'''
Computation of Spearmas Rank Correlation
@param  target: a pandas.dataframe (target) for wich we want to calculate the correlation matrix 
        possible_features: a list of pandas dataframes that are our features
        keys: a helping list, that manages the building_names and keeps track of the order in our possible_features 
        name: the building_name of the current dataframe we are predicting
@returns spearmans_matrix: a numpy.ndarray that contains the correlation value as first entry in each row and the corresponding p-value as a second element in each row. 
                            the order of stations in the matrix is consistent to the order in the dataset:
'''
def spearmansCorrelation (target, possible_features, name, keys = df_keys): 
    y_target = target['meter_reading']
    spearmans_matrix = np.zeros((len(possible_features), 2))
    for index, feature in enumerate(possible_features): 
        if ((target.shape[0] == feature.shape[0]) 
            and (keys[index] != (name, 'electricity_cleaned'))
            and ((feature['meter_reading'].isnull().sum()/feature.shape[0]) < 0.7)):
            
            # if shapes match, we are not at our target dataframe and the corresponding feature has sufficient data
            # we can compute the spearmans correlation
            correlation, pval = stats.spearmanr(y_target, feature['meter_reading'], nan_policy='omit')
            spearmans_matrix[index] = correlation, pval   

        else: 
            spearmans_matrix[index] = 0,100 
            continue
            
    return spearmans_matrix

In [ ]:
# Not needed any longer

# For given dataset (list of pd.Dataframes) we compute a subsample of those dataframes such that we obtain a list of dfs
# that do not fit the given building_name and are not of meter_type 'electricity_cleaned'
# Also we obtain the index/position of the dataframe that matches the building_id and is of type electricity_cleaned
def get_all_features(dataset, building_name):
    key_to_exclude = [building_name, 'electricity_cleaned']
    features = []
    index_of_df = np.nan
    for index, key in enumerate(group_keys):
        # as the key composes of two elements, the buildling_id and the meter_type we compare two arrays of strings, therefore we need the .all() 
        if(key == key_to_exclude).all():
            index_of_df = index
            continue
        features.append(dataset[index])
            
    return features, index_of_df

In [ ]:
# Can be moved closer to the heading

# Split a dataframe into train and test data according to a split_date


def split_df_to_train_and_test(df, split_date):
    df_train = df.loc[df['timestamp'] < split_date].copy().reset_index(drop=True) 
    df_test  = df.loc[df['timestamp'] >= split_date].copy().reset_index(drop=True)
    return df_train, df_test

# Computation of the Spearmans correlation matrix, with all other possible features

In [ ]:
buildingNames[14:]

In [ ]:
correlation_matrices = []
for name in tqdm(buildingNames[14:]): 
    print(name)
    
    # Get dataframe that matches our building_id
    df = complete_data_cleaned.loc[(complete_data_cleaned['building_id']  == name) 
                                   & (complete_data_cleaned['meter'] == 'electricity_cleaned') ]
    
    # For later reconstruction we split the electricity meter reading into train and test set
    y_train_df, y_test_df = split_df_to_train_and_test(df, SPLIT_TIMESTAMP)
    
    
    # Compute the spearmans correlation between all those features in order to pick the most relevant ones
    spearmans_matrix = spearmansCorrelation(y_train_df, dfs_train, name)
    
    
    with open ('../data/correlation_matrix_{}.pkl'.format(name), 'wb') as outp:
        pickle.dump(spearmans_matrix, outp, pickle.HIGHEST_PROTOCOL )
    
    correlation_matrices.append((spearmans_matrix, name))

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

Bear_utility_Sidney


 C:\Users\marku\anaconda3\lib\site-packages\scipy\stats\stats.py:4484: SpearmanRConstantInputWarning:An input array is constant; the correlation coefficient is not defined.
  6%|████▉                                                                          | 1/16 [18:40<4:40:13, 1120.88s/it]

Cockatoo_religion_Diedre


 12%|██████████                                                                      | 2/16 [25:57<2:47:36, 718.29s/it]

...

Lamb_industrial_Carla


 81%|████████████████████████████████████████████████████████████████▏              | 13/16 [2:45:54<42:12, 844.05s/it]

Peacock_lodging_Matthew


 88%|█████████████████████████████████████████████████████████████████████▏         | 14/16 [3:01:51<29:16, 878.02s/it]

Rat_public_Loretta


In [ ]:

with open ('../data/correlation_matrices.pkl', 'wb') as outp:
    pickle.dump(correlation_matrices, outp, pickle.HIGHEST_PROTOCOL )
    
        